In [ ]:
import polars as pl
def heat_pivot(df, values, index, columns, aggregate_function=None, cmap=None, xlabels=None, ylabels=None, annot_rescale=False, color_rescale=False):
    df = df.sort(columns)
    df = df.pivot(values=values, index=index, columns=columns, aggregate_function=aggregate_function)
    df = df.reverse()
    # df = df.select(df.columns[::-1])
    df_ind_ser= df.select(index).to_series()
    df = df.select(pl.all().exclude(index))

    if xlabels is None:
        xlabels = df.columns
    if ylabels is None:
        ylabels = df_ind_ser
    
    
    # heatmap用
    def rescale(df):
        for col_name in df.columns:
            col_ser = df[col_name]
            max_val = col_ser.max()
            min_val = col_ser.min()
            
            if max_val != min_val:
                df = df.with_columns(pl.col(col_name).sub(min_val).truediv(max_val - min_val))
        return df
    re_df = rescale(df)
    

    if annot_rescale:
        annot_C = re_df.to_numpy()
    else:
        annot_C = df.to_numpy()

    if color_rescale:
        color_C = re_df.to_numpy()
    else:
        color_C = df.to_numpy()
    
    annot_C *= 100


    sns.set_theme(rc={"figure.figsize": (len(xlabels)/1.5, len(ylabels)/1.5)})
    ax = sns.heatmap(color_C, annot=annot_C, cbar=True, square=True, cmap=cmap, xticklabels=xlabels, yticklabels=ylabels, fmt=".1f")
    ax.set_xlabel("# of data", fontsize=16)
    ax.set_ylabel("(fils, ensembles)", fontsize=14, rotation=90)
    plt.show()
    
    

for exp_dir in exp_dirs:
    rv = RunViewer(exp_path=exp_dir)
    # rv.write_results()
    
    df_stats = rv.fetch_results()
    df_stats_ext = df_stats

    df_stats_ext = df_stats_ext.with_columns((pl.col("fils") * pl.col("fils") * pl.col("ensembles")).alias("fi_en"))
    df_stats_ext = df_stats_ext.with_columns(pl.concat_str([pl.lit("("), pl.col("fils"), pl.lit(","), pl.col("ensembles"), pl.lit(")")]).alias("fe"))

    for fi_en in df_stats_ext["fi_en"].unique().sort():
        df_stats_fi = df_stats_ext.filter(pl.col("fi_en") == fi_en)
        # df_stats_ext = df_stats_ext.sort("fils")

        heat_pivot(df_stats_fi, values="val_acc", index="fe", columns="num_data", cmap="Blues_r", aggregate_function="mean", annot_rescale=False, color_rescale=True)


